<a href="https://colab.research.google.com/github/gRedDeer/kaggle_notebooks/blob/main/Loan_Approval_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Loan Approval Prediction/train.csv', index_col='id')
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Loan Approval Prediction/test.csv', index_col='id')
orig_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Loan Approval Prediction/credit_risk_dataset.csv')

# **Not sure why code is needed or was used to skipping it for later.**
```
# This is formatted as code
train_df = pd.concat([train_df, orig_df])
train_df = train_df.reset_index(drop=True)
train_df.duplicated().sum()
train_df.drop_duplicates(inplace=True)
train_df['loan_int_rate'] = train_df['loan_int_rate'].fillna(train_df['loan_int_rate'].mean())
train_df['person_emp_length'] = train_df['person_emp_length'].fillna(train_df['person_emp_length'].mean())
```



In [5]:
train_df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58645 entries, 0 to 58644
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  58645 non-null  int64  
 1   person_income               58645 non-null  int64  
 2   person_home_ownership       58645 non-null  object 
 3   person_emp_length           58645 non-null  float64
 4   loan_intent                 58645 non-null  object 
 5   loan_grade                  58645 non-null  object 
 6   loan_amnt                   58645 non-null  int64  
 7   loan_int_rate               58645 non-null  float64
 8   loan_percent_income         58645 non-null  float64
 9   cb_person_default_on_file   58645 non-null  object 
 10  cb_person_cred_hist_length  58645 non-null  int64  
 11  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 7.8+ MB


In [15]:
train_df.isna().sum()

,0
person_age,0
person_income,0
person_home_ownership,0
person_emp_length,0
loan_intent,0
loan_grade,0
loan_amnt,0
loan_int_rate,0
loan_percent_income,0
cb_person_default_on_file,0


In [17]:
target = train_df['loan_status']
train_df.drop('loan_status', axis=1, inplace=True)
features = train_df.copy()

In [25]:
cat_features = features.select_dtypes(include='object').columns.tolist()
num_features = features.select_dtypes(exclude='object').columns.tolist()

In [21]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,58645.0,27.550857,6.033216,20.00,23.00,26.00,30.00,123.00
person_income,58645.0,64046.172871,37931.106978,4200.00,42000.00,58000.00,75600.00,1900000.00
person_emp_length,58645.0,4.701015,3.959784,0.00,2.00,4.00,7.00,123.00
loan_amnt,58645.0,9217.556518,5563.807384,500.00,5000.00,8000.00,12000.00,35000.00
loan_int_rate,58645.0,10.677874,3.034697,5.42,7.88,10.75,12.99,23.22
loan_percent_income,58645.0,0.159238,0.091692,0.00,0.09,0.14,0.21,0.83
cb_person_cred_hist_length,58645.0,5.813556,4.029196,2.00,3.00,4.00,8.00,30.00


In [29]:
train_df[cat_features].describe(include='O').T

,count,unique,top,freq
person_home_ownership,58645,4,RENT,30594
loan_intent,58645,6,EDUCATION,12271
loan_grade,58645,7,A,20984
cb_person_default_on_file,58645,2,N,49943


In [36]:
for col in cat_features:
    display(train_df.groupby(col)[['loan_amnt', 'loan_int_rate']].agg(['min', 'mean', 'max']))

loan_amnt                      loan_int_rate             \
                            min          mean    max           min       mean   
person_home_ownership                                                           
MORTGAGE                   1000   9963.269417  35000          5.42   9.958439   
OTHER                      1000  10028.651685  25000          5.79  11.561348   
OWN                        1000   8740.431804  35000          5.42  10.707307   
RENT                        500   8659.063117  35000          5.42  11.256036   

                              
                         max  
person_home_ownership         
MORTGAGE               21.74  
OTHER                  20.52  
OWN                    20.11  
RENT                   23.22

loan_amnt                      loan_int_rate             \
                        min          mean    max           min       mean   
loan_intent                                                                 
DEBTCONSOLIDATION      1000   9150.314793  35000          5.42  10.613230   
EDUCATION              1000   9051.681363  35000          5.42  10.681510   
HOMEIMPROVEMENT        1000  10224.007962  35000          5.42  10.881809   
MEDICAL                 700   8819.990397  35000          5.42  10.742830   
PERSONAL               1000   9239.801318  35000          5.42  10.607075   
VENTURE                 500   9262.830886  35000          5.42  10.604354   

                          
                     max  
loan_intent               
DEBTCONSOLIDATION  21.74  
EDUCATION          21.74  
HOMEIMPROVEMENT    22.11  
MEDICAL            21.36  
PERSONAL           22.06  
VENTURE            23.22

loan_amnt                      loan_int_rate                  
                 min          mean    max           min       mean    max
loan_grade                                                               
A                700   8447.103746  35000          5.42   7.335176  19.04
B               1000   9622.485637  35000          5.99  11.034733  17.56
C                500   9009.894889  35000          5.79  13.510343  16.29
D               1000  10521.468017  35000          6.00  15.404599  19.13
E               1000  12176.313181  35000          7.88  16.930961  23.06
F               1750  12833.892617  30000         15.01  18.610940  22.06
G               1500  12559.090909  30000         18.07  20.327576  23.22

loan_amnt                     loan_int_rate  \
                                min         mean    max           min   
cb_person_default_on_file                                               
N                               500  9170.477985  35000          5.42   
Y                              1000  9487.752241  35000          5.99   

                                             
                                mean    max  
cb_person_default_on_file                    
N                          10.042178  21.74  
Y                          14.326300  23.22

Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
